In [1]:
import pandas as pd
import numpy as np
import os
import requests
from urllib.parse import urlparse
from urllib.parse import quote # this is used to fix illegal characters in the titlenames.
from sqlalchemy import create_engine

In [2]:
# Resaved the metacritic_movies.csv to get rid of commas in the numbers.
# Created schema for movies_t. Imported data into table just to make sure it worked, then deleted the table, and recreated
# to be an empty table.

In [3]:
# load the movies data.
csv_file = "Metacritic Reviews/metacritic_movies.csv"
raw_movies_df = pd.read_csv(csv_file)
raw_movies_df.head()

,movie_title,release_date,genre,meta_mixed,meta_negative,meta_positive,metascore,user_mixed,user_negative,user_positive,userscore
0,Anatomy of a Murder,1-Jul-59,"Drama,Mystery,Thriller,Crime",0,0,15,95,0,0,3,tbd
1,Bringing Up Baby,18-Feb-38,"Comedy,Romance,Family",0,1,16,91,1,0,2,tbd
2,After Life,12-May-99,"Drama,Fantasy",0,0,19,91,0,2,1,tbd
3,Gavagai,3-Aug-18,Drama,1,0,6,91,0,1,2,tbd
4,The Hustler,25-Sep-61,"Drama,Sport",1,0,17,90,0,0,3,tbd


In [4]:
# Reviews data cleaning
# Previewed in Excel.
# Removed lines of invalid data that appears to have wrapped incorrectly.

# Need to fix text for the apostrophes that have been convered to â€™
# Did Find/Replace using text editor.
# 
# created file Cleaned_metacritic_reviews.csv

In [5]:
# Load the reviews data.
csv_file = "Metacritic Reviews/Cleaned_metacritic_reviews.csv"
raw_reviews_df = pd.read_csv(csv_file, dtype={3:'str',4:'str'})
raw_reviews_df.head()

,critic_name,media,movie_title,review_date,individual_meta_score,text
0,Robbie Collin,The Telegraph,Jojo Rabbit,7-Oct-19,20,As satire it's a dismal dereliction of duty; a...
1,Roger Moore,NaN,Carrie Pilby,7-Oct-19,38,Carrie Pilby never rises to the level of hatef...
2,NaN,Empire,Gemini Man,7-Oct-19,40,It gives you two Will Smiths for the price of ...
3,Noel Murray,Los Angeles Times,In the Tall Grass,7-Oct-19,50,Despite the handsome Craig Wrobleski cinematog...
4,Jesse Hassenger,The A.V. Club,The King,7-Oct-19,50,It's hard to feel energized by a historical ep...


In [6]:
## Cleanup complete

In [7]:
## Addy's section

# Need to create table of movie posters urls from TheMovieDB.org
#
# Choice 1: Lookup table
#    Movie name, poster url

# Choice 2: Append a new column for poster url to movies_df

In [8]:
#url = 'https://api.themoviedb.org/3/movie/550?api_key=3d48b9204ca053957eb4375bb97a83c8'

In [9]:
#keep
config_path = 'http://api.themoviedb.org/3/configuration?api_key={key}'
KEY = '3d48b9204ca053957eb4375bb97a83c8'
url = config_path.format(key=KEY)
r = requests.get(url)
config = r.json()
# config

In [10]:
#keep
base_url = config['images']['base_url']
base_url

'http://image.tmdb.org/t/p/'

In [11]:
# This is what happens if there's no poster.

temp = 'http://api.themoviedb.org/3/search/movie?query="Besotted"&api_key=3d48b9204ca053957eb4375bb97a83c8'
r = requests.get(temp)
response = r.json()
if response['results'][0]['poster_path'] is None:
    print('none')

none


In [12]:
# This is what happens if the movie title is not found.

temp = 'http://api.themoviedb.org/3/search/movie?query="dasgdsf"&api_key=3d48b9204ca053957eb4375bb97a83c8'
r = requests.get(temp)
response = r.json()
response['total_results']

0

In [13]:
# This is what happens if the movie title has # in it.

temp = 'http://api.themoviedb.org/3/search/movie?query="#Horror"&api_key=3d48b9204ca053957eb4375bb97a83c8'
r = requests.get(temp)
response = r.json()
response

{'status_code': 7,
 'status_message': 'Invalid API key: You must be granted a valid key.',
 'success': False}

In [14]:
# This is what happens if the movie title has # in it.

movieTitle=quote('#Horror')
temp = 'http://api.themoviedb.org/3/search/movie?query=' + movieTitle + '&api_key=3d48b9204ca053957eb4375bb97a83c8'
print(temp)
#r = requests.get(temp)
#response = r.json()
#response

http://api.themoviedb.org/3/search/movie?query=%23Horror&api_key=3d48b9204ca053957eb4375bb97a83c8


In [15]:
# not sure why this movie is one that has the values in the tuple swapped.

movieTitle=quote('No Home Movie')
temp = 'http://api.themoviedb.org/3/search/movie?query=' + movieTitle + '&api_key=3d48b9204ca053957eb4375bb97a83c8'
#print(temp)
r = requests.get(temp)
response = r.json()
response['results'][0]['poster_path']

'/eUL2USo8weZHhNMJuWwKtegJzhL.jpg'

In [16]:
# Reused
# base_url = 'http://image.tmdb.org/t/p/'
# size = 'w500'
# rel_path = '/mgwZvl0DwS1GIborSGlpTGomrJD.jpg'

In [17]:
base_url = 'http://image.tmdb.org/t/p/'
size = 'w500' # We're looking for 500px sized posters

In [18]:
url_list = []

# the following 2 lists aren't needed since we're just writing them to a file.
#movie_not_found_list = []
#poster_not_found_list = []

processed_count = len(raw_movies_df)
movies_not_found_count = 0
posters_not_found_count = 0
keyerrors_count = 0

keyerrors_f = open("keyerrors.txt", "w")
movies_not_found_f = open("movies_not_found.txt", "w")
posters_not_found_f = open("posters_not_found.txt", "w")
url_list_f = open("url_list.txt", "w")

print('-----------------------------')
print('Begin Data Retreival. Rows left.')
print('-----------------------------\n')
for title in raw_movies_df.movie_title:
    #print(title)
    temp = 'http://api.themoviedb.org/3/search/movie?query=' + quote(title) + '&api_key=3d48b9204ca053957eb4375bb97a83c8'
    #print(temp)
    try:
        r = requests.get(temp)
        response = r.json()
        
        if (response['total_results'] != 0) and (response['results'][0]['poster_path'] is not None):
            url = base_url + size + response['results'][0]['poster_path']
            url_list.append({title, url})
            url_list_f.write(title + "\t" + url + "\n") # make it Tab delimited instead.
        elif response['total_results'] == 0:
            #error_list.append(f'{title} not found. Skipping...')
            #movie_not_found_list.append(title)
            movies_not_found_f.write(title + '\n')
            movies_not_found_count += 1
        elif response['results'][0]['poster_path'] is None:
            #print(f'No poster found for {title}.')
            #poster_not_found_list.append(title)
            posters_not_found_f.write(title + '\n')
            posters_not_found_count += 1
        
        # Commmented out as this was printing out the complete list each time... 
        # print(url_list)
        
        #read json into a list
        #movie_url.append(response_json, [url])
        
        #Print a period every iteration so that we know this is still running
        processed_count -= 1
        if (processed_count % 25 == 0):
            print(processed_count, end = '') # print the number left on every count divisable by 25.
        print('.', end = '')
        
    except KeyError:
        print(f'\n ** keyerror with {title} ** \n')
        keyerrors_count += 1
        keyerrors_f.write(title + '\n')
    
# close the files.
url_list_f.close()
keyerrors_f.close()
movies_not_found_f.close()
posters_not_found_f.close()

print('\n\n-----------------------------')
print('Data Retrieval Complete')
print('-----------------------------')
print(f'Movies not found: {movies_not_found_count}')
print(f'Movies with no posters: {posters_not_found_count}')
print(f'Other errors: {keyerrors_count}\n')
print('See the files movies_not_found.txt, posters_not_found.txt, and keyerrors.txt for details.')      
    

-----------------------------
Begin Data Retreival. Rows left.
-----------------------------



....................9100.........................9075.........................9050.........................9025.........................9000.........................8975.........................8950.........................8925.........................8900.........................8875.........................8850.........................8825.........................8800.........................8775.........................8750.........................8725.........................8700.........................8675.........................8650.........................8625.........................8600.........................8575.........................8550.........................8525.........................8500.........................8475.........................8450.........................8425.........................8400.........................8375.........................8350.........................8325.........................8300.........................8275...................

.....2050.........................2025.........................2000.........................1975.........................1950.........................1925.........................1900.........................1875.........................1850.........................1825.........................1800.........................1775.........................1750.........................1725.........................1700.........................1675.........................1650.........................1625.........................1600.........................1575.........................1550.........................1525.........................1500.........................1475.........................1450.........................1425.........................1400.........................1375.........................1350.........................1325.........................1300.........................1275.........................1250.........................1225.........................1200.....

In [19]:
# url_list

In [20]:
url_df = pd.DataFrame(url_list, columns = ['movie_title','poster_url'])
url_df.tail(5)

,movie_title,poster_url
8738,http://image.tmdb.org/t/p/w500/vIb9kzseUun3QGo...,Wild Reeds
8739,No Home Movie,http://image.tmdb.org/t/p/w500/eUL2USo8weZHhNM...
8740,http://image.tmdb.org/t/p/w500/yRMex0csH3oI05Y...,Solas
8741,http://image.tmdb.org/t/p/w500/43ffZhMCWQhzMne...,J.T. Leroy
8742,http://image.tmdb.org/t/p/w500/jcuEDKnM3uThdz7...,Donnybrook


In [21]:
# No clue why the title and url would get swapped like that!! :-/


In [22]:
# Do we still need to do this?

# result = pd.concat([movie_title_df, url_df],axis= , sort=True)

In [23]:
# reference
# for title in raw_movies_df.movie_title:
#     #print(title)
#     temp = 'http://api.themoviedb.org/3/search/movie?query="' + title + '"&api_key=3d48b9204ca053957eb4375bb97a83c8'
#     #print(temp)
#     r = requests.get(temp)
#     response = r.json()
#     url = base_url + size + response['results'][0]['poster_path']
#     print(url)

In [24]:
# let's see if we can pull up a movie poster.

